In [108]:
import pandas as pd
import numpy as np
from classifier import classifier

In [109]:
df = pd.read_csv("adult.data.csv",header=None)
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income']
test_df =  pd.read_csv("adult.test.csv",header=None)
test_df.columns = df.columns

In [110]:
#drop unknown data
df = df[(df.workclass != '?')]
df = df[(df['native.country'] != '?')]
df = df[(df.occupation != '?')]

#drop unknown data
test_df = test_df[(test_df.workclass != '?')]
test_df = test_df[(test_df['native.country'] != '?')]
test_df = test_df[(test_df.occupation != '?')]

In [111]:
from sklearn import preprocessing

workclass_le = preprocessing.LabelEncoder()
workclass_le.fit(df['workclass'])
df['workclass'] = workclass_le.transform(df['workclass'])
test_df['workclass'] = workclass_le.transform(test_df['workclass'])

education_le = preprocessing.LabelEncoder()
education_le.fit(df['education'])
df['education'] = education_le.transform(df['education'])
test_df['education'] = education_le.transform(test_df['education'])

marital_le = preprocessing.LabelEncoder()
marital_le.fit(df['marital.status'])
df['marital.status'] = marital_le.transform(df['marital.status'])
test_df['marital.status'] = marital_le.transform(test_df['marital.status'])

occupation_le = preprocessing.LabelEncoder()
occupation_le.fit(df['occupation'])
df['occupation'] = occupation_le.transform(df['occupation'])
test_df['occupation'] = occupation_le.transform(test_df['occupation'])

relation_le = preprocessing.LabelEncoder()
relation_le.fit(df['relationship'])
df['relationship'] = relation_le.transform(df['relationship'])
test_df['relationship'] = relation_le.transform(test_df['relationship'])

race_le = preprocessing.LabelEncoder()
race_le.fit(df['race'])
df['race'] = race_le.transform(df['race'])
test_df['race'] = race_le.transform(test_df['race'])

sex_le = preprocessing.LabelEncoder()
sex_le.fit(df['sex'])
df['sex'] = sex_le.transform(df['sex'])
test_df['sex'] = sex_le.transform(test_df['sex'])

native_le = preprocessing.LabelEncoder()
native_le.fit(df['native.country'])
df['native.country'] = native_le.transform(df['native.country'])
test_df['native.country'] = native_le.transform(test_df['native.country'])

income_le = preprocessing.LabelEncoder()
income_le.fit(df['income'])
df['income'] = income_le.transform(df['income'])

income_le_test = preprocessing.LabelEncoder()
income_le_test.fit(test_df['income'])
test_df['income'] = income_le_test.transform(test_df['income'])

In [112]:
df['age_trans'] = pd.cut(df['age'],3,labels=np.array([0,1,2]))
df['hours.per.week_trans'] = pd.cut(df['hours.per.week'],3,labels=np.array([0,1,2]))
df['fnlwgt_trans'] = pd.cut(df['fnlwgt'],3,labels=np.array([0,1,2]))
df['capital.gain_trans'] = pd.cut(df['capital.gain'],3,labels=np.array([0,1,2]))
df['capital.loss_trans'] = pd.cut(df['capital.loss'],3,labels=np.array([0,1,2]))

In [113]:
series_age,bin_age = pd.cut(df['age'],3,labels=False,retbins=True)
series_hpw,bin_hpw = pd.cut(df['hours.per.week'],3,labels=False, retbins=True)
series_fnlwgt,bin_fnlwgt = pd.cut(df['fnlwgt'],3,labels=False, retbins=True)
series_capital_gain,bin_capital_gain = pd.cut(df['capital.gain'],3,labels=False, retbins=True)
series_capital_loss,bin_capital_loss = pd.cut(df['capital.loss'],3,labels=False, retbins=True)

In [114]:
test_df['age_trans'] = pd.cut(test_df['age'],bins=bin_age,labels=np.array([0,1,2]))
test_df['hours.per.week_trans'] = pd.cut(test_df['hours.per.week'],bins=bin_hpw,labels=np.array([0,1,2]))
test_df['fnlwgt_trans'] = pd.cut(test_df['fnlwgt'],bins=bin_fnlwgt,labels=np.array([0,1,2]))
test_df['capital.gain_trans'] = pd.cut(test_df['capital.gain'],bins=bin_capital_gain,labels=np.array([0,1,2]))
test_df['capital.loss_trans'] = pd.cut(test_df['capital.loss'],bins=bin_capital_loss,labels=np.array([0,1,2]))

In [115]:
Y_train = df.iloc[:,-1]
Y_test = test_df.iloc[:,-1]
X_train = df[['age_trans', 'workclass', 'fnlwgt_trans', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain_trans', 'capital.loss_trans', 'hours.per.week_trans', 'native.country']]
X_test = df[['age_trans', 'workclass', 'fnlwgt_trans', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain_trans', 'capital.loss_trans', 'hours.per.week_trans', 'native.country']]

In [122]:
from decision_tree import decision_tree

class random_forest(classifier):
    
    def __init__(self,trees=10, max_depth=-1):
        self.n_trees = trees
        self.max_depth = max_depth
        self.trees_list = None
       
    def fit(self,X,Y):
        import pandas as pd
        
        dataset_train = pd.concat([X, Y], axis=1)
        sample_size = 1.0
        feature_size = int(np.sqrt(len(X[0])-1))
        trees = list()
        
        for i in range(self.n_trees):
            dtree_clf = decision_tree()
            features = self.subfeatures(X.columns,feature_size)
            sample_X, sample_Y = self.subsample(dataset_train[features],sample_size)
            dtree_clf.fit(sample_X.values.tolist(),sample_Y.values.tolist())
            trees.append(dtree_clf)
        self.trees_list = trees
        return self.trees_list

        
    def predict (self,X_test):
        import operator
        from collections import defaultdict
        
        predictions = [t.predict(X_test.values.tolist()) for t in self.trees_list]
        hypothesis = []
        for x in range(0,len(X_test)):
            count =  defaultdict(int)
            for p in predictions:
                if p[x] != None:
                    count[p[x]] +=1
            max_vote = max(count.items(), key=operator.itemgetter(1))[0]
            hypothesis.append(max_vote)
        return hypothesis
        
    
    def subsample(self,dataset,ratio):
        import numpy as np
        sample = list()
        n_sample = round(len(dataset) * ratio)
        chosen_idx=np.random.choice(len(dataset), replace=False, size=n_sample)
        dataset_trimmed =dataset.iloc[chosen_idx]
        sample_X = dataset_trimmed.iloc[:,:-1]
        sample_Y = dataset_trimmed.iloc[:,-1]
        return sample_X, sample_Y
    
    def subfeatures(self,features, n_sample):
        import numpy as np
        return np.random.choice(features,n_sample,replace=False)

In [123]:
clf = random_forest(max_depth=5)
fitted_data = clf.fit(X_train,Y_train)

KeyError: 0

In [ ]:
hyp = clf.predict(X_test)

In [ ]:
def accuracy_score(labels, hypotheses):
    count = 0.0
    correct = 0.0

    for l, h in zip(labels, hypotheses):
        count += 1.0
        if l == h:
            correct += 1.0
    return correct / count

In [ ]:
accuracy_score(Y_test,hyp)